<a href="https://colab.research.google.com/github/NanMinuk/UOS_AI/blob/master/TF_IDF%2BLR%2CRF%2CXgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q KoNLPy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.3 MB/s eta 0:00:00


# 사용할 라이브러리 import

In [2]:
import numpy as np
import pandas as pd
import re
import time
import matplotlib.pyplot as plt

from konlpy.tag import Okt

# 사용할 train용 데이터

In [3]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Github/UOS_NLP/ratings_train.txt", sep='\t')
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [4]:
print(train_df['label'].value_counts())
print(train_df.info())

0    75173
1    74827
Name: label, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
None


# 사용할 테스트용 데이터

In [5]:
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Github/UOS_NLP/ratings_test.txt',sep='\t')
test_df

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [6]:
print(test_df['label'].value_counts())
print(test_df.info())

1    25173
0    24827
Name: label, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB
None


# NULL 데이터 제거

In [7]:
print("<train data null count>")
print(train_df.isna().sum())
print("<test data null count>")
print(test_df.isna().sum())

<train data null count>
id          0
document    5
label       0
dtype: int64
<test data null count>
id          0
document    3
label       0
dtype: int64


In [8]:
train_df=train_df.fillna(" ")
test_df=test_df.fillna(" ")

In [9]:
okt = Okt()

# 문자 전처리

In [14]:
train_df['document'] = train_df['document'].apply(lambda x: re.sub('\\\\n', ' ',x) ) # 개행문자 제거
train_df['document'] = train_df['document'].apply(lambda x: re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ ]', '',x ) )# 개행문자 제거


In [15]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


# 필요없는 id 컬럼 제거

In [16]:
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

# 벡터화에 쓰일 tokenizer 함수 정의

In [19]:
def tw_tokenizer(sentence,remove_stop_words=True):

  stop_words = set(['에', '은', '는', '이', '가', '그리고', '것', '들', '수', '등', '로', '을', '를', '만', '도', '아', '의', '그', '다'])
  kor_tokens = okt.morphs(sentence,stem=True)
  if remove_stop_words:
        kor_tokens = [token for token in kor_tokens if not token in stop_words]

  return kor_tokens

# 벡터화에 필요한 라이브러리

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# TF_IDF 적용

In [22]:
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [23]:
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

In [24]:
print(tfidf_matrix_train.shape)

(150000, 93964)


In [25]:
print(tfidf_matrix_test.shape)

(50000, 93964)


# Logistic Regression

In [26]:
lg_clf = LogisticRegression(random_state=0, solver='liblinear')

In [27]:
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )
grid_cv.fit(tfidf_matrix_train , train_df['label'] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))


Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8563


# Linear Regression 최종 평가 지표

In [30]:
from sklearn.metrics import accuracy_score, precision_score, f1_score
best_lr_estimator = grid_cv.best_estimator_
preds = best_lr_estimator.predict(tfidf_matrix_test)

print('Logistic Regression accuracy_score:', accuracy_score(test_df['label'],preds))
print('Logistic Regression precision_score:', precision_score(test_df['label'],preds))
print('Logistic Regression f1_score:', f1_score(test_df['label'],preds))

Logistic Regression accuracy_score: 0.85994
Logistic Regression precision_score: 0.8668631884994347
Logistic Regression f1_score: 0.8597633017602179


# Random Forest

In [34]:
rf_clf = RandomForestClassifier(n_estimators = 100, random_state =0)

In [35]:
params = {
    'max_depth': [8, 16, 24],
    'min_samples_leaf' : [1, 6, 12],
    'min_samples_split' : [2, 8, 16]
}
grid_cv = GridSearchCV(rf_clf,param_grid = params, cv=2)
grid_cv.fit(tfidf_matrix_train,train_df['label'])

print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

{'max_depth': 24, 'min_samples_leaf': 1, 'min_samples_split': 16} 0.7925


# Random Forest 최종 평가 지표

In [37]:
best_rf_estimator = grid_cv.best_estimator_
preds = best_rf_estimator.predict(tfidf_matrix_test)

print('Random Forest accuracy_score:', accuracy_score(test_df['label'],preds))
print('Random Forest precision_score:', precision_score(test_df['label'],preds))
print('Random Forest f1_score:', f1_score(test_df['label'],preds))

Random Forest accuracy_score: 0.79202
Random Forest precision_score: 0.8272058823529411
Random Forest f1_score: 0.7822153343525517


# XGBoost

In [52]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

# 검증용 데이터 추가

In [53]:
X_tr, X_val, y_tr, y_val= train_test_split(tfidf_matrix_train, train_df['label'], test_size=0.1, random_state=0 )
print(X_tr.shape, X_val.shape)

(135000, 93964) (15000, 93964)


# Dmartix -> XGBoost input 형식

In [54]:
dtr = xgb.DMatrix(data=X_tr, label=y_tr)
dval = xgb.DMatrix(data=X_val, label=y_val)
dtest = xgb.DMatrix(data=tfidf_matrix_test , label=test_df['label'])

In [55]:
params = { 'max_depth':3,
           'eta': 0.05,
           'objective':'binary:logistic',
           'eval_metric':'logloss',
           'learning_rate': 0.01
        }
num_rounds = 4000

eval_list = [(dtr,'train'),(dval,'eval')]

In [ ]:
xgb_model = xgb.train(params = params , dtrain=dtr , num_boost_round=num_rounds , \
                      early_stopping_rounds=50, evals=eval_list )

In [57]:
 pred_probs = xgb_model.predict(dtest)
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]

In [60]:
print(preds[:10])

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]


# XGBoost 최종 평가 지표

In [59]:
print('XGBoost accuracy_score:', accuracy_score(test_df['label'],preds))
print('XGBoost precision_score:', precision_score(test_df['label'],preds))
print('XGBoost f1_score:', f1_score(test_df['label'],preds))

XGBoost accuracy_score: 0.78508
XGBoost precision_score: 0.8374421106797025
XGBoost f1_score: 0.7691514500537057
